# Refs & Nexus Database


One of the core features of Quantinuum Nexus is a database for usefully structuring your scientific data.

`qnexus` aims to provide a useful way of interacting with this database through:

- `Ref` objects that can be used as a proxy object to 'reference' data like Circuits or Projects.

- `get` functions that take filters and try to return a single matching `Ref` 

- `get_all` functions that take filters and return `NexusIterator` objects for more than one matching`Ref`.  

In [2]:
from datetime import datetime, timedelta

import qnexus as qnx

from pytket import Circuit

In [4]:
# Get a 'ProjectRef' to a Project in Nexus, creating it if it doesn't exist
my_project_ref = qnx.projects.get_or_create(name="My Project")

In [5]:
# Refs have a pandas DataFrame representation
my_project_ref.df()

,name,description,created,modified,contents_modified,id
0,My Project,None,2024-07-18 17:41:39.914467+00:00,2024-07-18 17:41:39.914467+00:00,2024-07-18 17:41:39.917983+00:00,7425c855-a865-40db-aadf-b4d5f907e0f6


In [6]:
# Upload a circuit to Nexus, getting a CircuitRef in return
my_circuit_ref = qnx.circuits.upload(
    name=f"My Circuit from {datetime.now()}",
    circuit = Circuit(2).H(0).CX(0,1).measure_all(),
    project = my_project_ref,
)

my_circuit_ref.df()

,name,description,created,modified,project,id
0,My Circuit from 2024-07-18 18:41:51.500787,None,2024-07-18 17:41:52.016706+00:00,2024-07-18 17:41:52.016706+00:00,My Project,416c7574-9630-43a7-a989-6b0b8244aa17


Refs point to underlying scientific data, in this case a pytket Circuit. We need to call a function to 'dereference' the CircuitRef and obtain the underlying circuit.

In [7]:
my_circuit_ref.download_circuit()

[H q[0]; CX q[0], q[1]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; ]

We can perform queries to Nexus to retrieve all matching references.

In [8]:
last_months_projects = qnx.projects.get_all(created_after=datetime.now() - timedelta(days=30))

This will return a NexusIterator, a Python object that represents our query. It behaves like a Python Iterator but with some additional functionality.

In [9]:
# See how many projects match my query
last_months_projects.count()

501

In [10]:
# Get the first ProjectRef
next(last_months_projects)

ProjectRef(id=UUID('0723e0d8-6583-4818-be31-577142fcf588'), annotations=Annotations(name='qnexus_integration_test_project_2024-06-19 12:03:35.275984', description='This can be safely deleted. Test Run: 2024-06-19 12:03:36.153727', properties=OrderedDict(), created=datetime.datetime(2024, 6, 19, 12, 3, 36, 926667, tzinfo=TzInfo(UTC)), modified=datetime.datetime(2024, 6, 19, 12, 3, 36, 926667, tzinfo=TzInfo(UTC))), contents_modified=datetime.datetime(2024, 6, 19, 12, 18, 14, 367073, tzinfo=TzInfo(UTC)), type='ProjectRef')

You can use other methods on the NexusIterator to get the ProjectRefs.

```python
# Get all the projects as a list
last_months_projects.list()

# Get all the projects as a DataFrame
last_months_projects.df()
```

If we think we can identify a single matching Ref to our query, we can use `get`. Bear in mind this will error if there is not exactly one matching Ref.

In [4]:
qnx.projects.get(name_like="My Project").df()

,name,description,created,modified,contents_modified,id
0,My Project,None,2024-07-18 17:41:39.914467+00:00,2024-07-18 17:41:39.914467+00:00,2024-07-18 17:41:52.025227+00:00,7425c855-a865-40db-aadf-b4d5f907e0f6
